# Phase estimation algorithm

Steps:

- Hadamard wall
- controlled rotations
- inverse QFT

In [1]:
from qat.lang.AQASM.qftarith import IQFT
from qat.lang.AQASM import H, QRoutine, RX, Program

def build_pea_routine(U_routine, nbits_phase):
    """
    Args:
        U_routine (QRoutine): the unitary matrix 
        nbits_phase (int): number of bits for phase register
    """
    nbits_data = U_routine.arity
    
    routine = QRoutine()
    
    for qb in range(nbits_phase):
        routine.apply(H, qb)
    
    for j_ind in range(nbits_phase):
        for _ in range(2**j_ind):
            qbits = [j_ind] + [qb for qb in range(nbits_phase, nbits_phase + nbits_data)]
            routine.apply(U_routine.ctrl(), *qbits)
    
    qbits = list(range(nbits_phase))
    routine.apply(IQFT(nbits_phase), *qbits)
    
    return routine

## A simple example: a rotation, $U = R_z$

Let us consider
$$U = R_z(\theta) = e^{-i \theta / 2 \sigma_z}$$

In [11]:
import numpy as np

alpha = 0.32
# thus should readout theta = alpha / (4pi) = 0.32 / (4 pi) ~ 0.02546
# infact: 0.02546 * 2**nbits = 0.02546 * 2**2 = 

nbits_phase = 10
theta_exp = alpha/(4*np.pi) #* 2**nbits_phase
print("Expected output = %s (binary: %s)"%(theta_exp,
                                           bin(int(theta_exp* 2**nbits_phase))))



Expected output = 0.025464790894703257 (binary: 0b11010)


In [4]:
myroutine = QRoutine()
myroutine.apply(RX(alpha), 0)

pea_routine = build_pea_routine(myroutine, nbits_phase)

prog = Program()
reg = prog.qalloc(nbits_phase + myroutine.arity)
prog.apply(pea_routine, reg)

circ = prog.to_circ()

%jsqatdisplay circ

from qat.qpus import get_default_qpu
qpu = get_default_qpu()
res = qpu.submit(circ.to_job()) # could modify job
for sample in res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

|00000110100> 0.24597524918640126
|00000110101> 0.24460992034343623
|11111001100> 0.2459752491864012
|11111001101> 0.24460992034343623


Note: last qubit is data qubit

Why are there also states 1111... ?

Better: measure only phase register

In [5]:
res = qpu.submit(circ.to_job(qubits=list(range(nbits_phase)))) # could modify job
for sample in res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)

|0000011010> 0.49058516952983755
|1111100110> 0.49058516952983744


In [13]:
other_sol = int('1111100110', 2)/2**nbits_phase
print("Other solution:", other_sol)

theta_exp = 1 - alpha/(4*np.pi) #* 2**nbits_phase
print("Other alpha:", theta_exp)

Other solution: 0.974609375
Other alpha: 0.9745352091052968


### Refinement: typed registers

In [16]:
myroutine = QRoutine()
myroutine.apply(RX(alpha), 0)

pea_routine = build_pea_routine(myroutine, nbits_phase)

prog = Program()
phase_reg = prog.qalloc(nbits_phase, reg_type="FLOAT")
data_reg = prog.qalloc(myroutine.arity)
prog.apply(pea_routine, phase_reg, data_reg)

circ = prog.to_circ()


from qat.qpus import get_default_qpu
qpu = get_default_qpu()
res = qpu.submit(circ.to_job()) # could modify job
for sample in res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)

|0.025390625>|0> 0.24597524918640126
|0.025390625>|1> 0.24460992034343623
|0.974609375>|0> 0.2459752491864012
|0.974609375>|1> 0.24460992034343623


## Finding the ground state energy of a Hamiltonian